In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Lambda
from tensorflow.keras.backend import expand_dims, repeat_elements
import numpy as np
import h5py

In [15]:
class DataGenerator(Sequence):
    """Generates data for Keras"""

    def __init__(self, paths, batch_size=16, dim=(64, 64),
                 shuffle=True, k=16):
        """Initialization"""
        self.dim = dim
        self.file_batch_size = 1024
        self.batch_size = batch_size
        self.n_batches = self.file_batch_size // self.batch_size
        self.file_paths = paths
        self.shuffle = shuffle
        self.current_file_loaded = (None, None)
        self.k = k
        self.indexes = np.arange(len(self.file_paths) * int(self.file_batch_size / self.batch_size))
        self.on_epoch_end()

    def load_file(self, i):
        with h5py.File(self.file_paths[i], "r") as f:
            self.current_file_loaded = i, f["data/data"][:]

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.file_paths) * self.n_batches
        # TODO: Arreglar esto

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        file_index = index // self.n_batches
        current_file_index, _ = self.current_file_loaded
        if file_index != current_file_index:
            self.load_file(file_index)
        _, current_data = self.current_file_loaded
        index_inside_file = index % self.n_batches
        i = index_inside_file * self.batch_size
        data = current_data[i:(i + self.batch_size)]
        data[data > 1.0] = 1.0
        X = data[:, :self.k]
        y = data[:, self.k:self.k*2]
        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            np.random.shuffle(self.file_paths)
            self.current_file_loaded = (None, None)

In [ ]:
repeat_elements

In [3]:
from tensorflow.keras.losses import binary_crossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

def get_compiled_model():

    inp = layers.Input(shape=(32, 64, 64, 1))

    # Probar normalizar dividiendo entre 300
    # Probar normalizar log(x+1)
    
    x = layers.ConvLSTM2D(
        filters=48,
        kernel_size=(5, 5),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(inp)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        #padding="same",
        activation="relu",
    )(x)
    x = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), 16, 1))(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.Conv2D(
        filters=64, kernel_size=(3, 3), activation="sigmoid", padding="same"
    )(x)


    model = Model(inp, x)
    model.compile(
        loss=BinaryCrossentropy(), 
        optimizer=Adam(),
    )
    return model


In [4]:
# [0, 1>
# [1, 5>
# [5, ...>
# [..., 80>
# [80, inf>

In [5]:
model = get_compiled_model()

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 64, 64, 1)]   0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 32, 60, 60, 48)    235392    
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 58, 58, 64)        258304    
                                                                 
 lambda (Lambda)             (None, 16, 58, 58, 64)    0         
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 16, 58, 58, 64)    33024     
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 16, 58, 58, 64)    33024     
                                                                 
 conv2d (Conv2D)             (None, 16, 58, 58, 64)    36928 